# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [ ]:
# your code here
import pandas as pd
df=pd.read_csv("tic-tac-toe.csv")
print(df.head())

In [ ]:
# your code here
missing_data = df.isnull().sum()
print(missing_data)

# Check the percentage of missing data for each column
missing_percentage = (df.isnull().sum() / len(df)) * 100
print(missing_percentage)

In [ ]:
print(df.dtypes)

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
for column in df.select_dtypes(include=['object']).columns:
    df[column] = label_encoder.fit_transform(df[column])

print(df)

In [ ]:
print(df.dtypes)

In [ ]:
X = df.drop('class', axis=1)  # 'target_column' is the name of the output column
y = df['class']  # 'target_column' is the name of the output column

# Now X contains the inputs, and y contains the output
print("Features (X):")
print(X.head())

print("Labels (y):")
print(y.head())

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

# Fit the scaler to the data and transform it
X_normalized = scaler.fit_transform(X)

# Convert the result back into a DataFrame (optional)
X_normalized_df = pd.DataFrame(X_normalized, columns=X.columns)

y_encoded = pd.get_dummies(y)
# Show the normalized data
print(X_normalized_df.head())

## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [ ]:
# your code here
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y_encoded, test_size=0.2, random_state=42)

In [ ]:
from tensorflow.keras.models import Sequential
model = Sequential()

In [ ]:
from tensorflow.keras.layers import Dense
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))

# Add the second hidden layer with 64 neurons and ReLU activation
model.add(Dense(64, activation='relu'))

# Add the third hidden layer with 32 neurons and ReLU activation
model.add(Dense(32, activation='relu'))

# Add the output layer with as many neurons as unique classes in the target variable
model.add(Dense(y_encoded.shape[1],activation='softmax'))

In [ ]:
from sklearn.metrics import accuracy_score
import numpy as np
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=10, verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

# Make predictions on the test data
y_pred = model.predict(X_test)

# Convert predictions to class labels
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test.to_numpy(), axis=1)

# Calculate the accuracy score
accuracy = accuracy_score(y_true_classes, y_pred_classes)
print(f"Accuracy Score: {accuracy}")

In [ ]:
import joblib
joblib.dump(model, 'tic-tac-toe.model')

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [ ]:
# your code here
import joblib
model = joblib.load('tic-tac-toe.model')

In [ ]:
print(f"X_test type: {type(X_test)}")
print(f"y_test type: {type(y_test)}")

# Ensure the indices are valid for X_test and y_test
random_indices = np.random.choice(X_test.shape[0], size=5, replace=False)  # Select 5 random rows
print("Random indices selected:", random_indices)

# Extract the corresponding rows
X_random = X_test[random_indices]

# If y_test is a DataFrame, use .iloc[] for row-based indexing
# Alternatively, convert y_test to a numpy array to avoid issues
y_random = y_test.iloc[random_indices].to_numpy()  # Convert to numpy array for consistency

# Make predictions on the random rows
y_pred = model.predict(X_random)

# Output the predicted classes and the corresponding true labels
for i in range(len(random_indices)):
    predicted_class = np.argmax(y_pred[i])  # Convert prediction probabilities to the class with max probability
    true_class = np.argmax(y_random[i])  # True class for the random row
    print(f"Prediction for row {random_indices[i]}: {predicted_class}, True label: {true_class}")


## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [ ]:
# your code here
# add  more layers:
from tensorflow import keras
from tensorflow.keras import layers
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(9,)),  # Input layer
    layers.Dense(128, activation='relu'),  # Added hidden layer
    layers.Dense(64, activation='relu'),   # Another hidden layer
    layers.Dense(1, activation='sigmoid')  # Output layer (for binary classification)
])

In [ ]:
#Adjust the Learning Rate:
import tensorflow as tf
learning_rate = 0.001  # You can experiment with different values
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# Compile the model
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',  # Use categorical_crossentropy for multi-class
              metrics=['accuracy'])

In [ ]:
#Train for More Epochs
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
history = model.fit(X_train, y_train, epochs=50, validation_data=(X_val, y_val))

**Which approach(es) did you find helpful to improve your model performance?**

In [ ]:
# your answer here
#adding more layers was helpful to improve my model performance